<a href="https://colab.research.google.com/github/greyhound101/Orbuculum/blob/main/handeled_df_roi_resize_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Mounted at /content/gdrive


In [ ]:
import zipfile
import pandas as pd
import glob
import os
import cv2
from tqdm import tqdm
import random

In [2]:

with zipfile.ZipFile("/content/gdrive/MyDrive/GTSRB_Final_Training_Images.zip" , 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [4]:

cols=['Filename','Width','Height','Roi.X1','Roi.Y1','Roi.X2','Roi.Y2','ClassId']
for name in glob.glob('/content/GTSRB/Final_Training/Images/*/GT-*.csv'):
  df=pd.read_csv(name)
  dk={'Filename':[],'Width':[],'Height':[],'Roi.X1':[],'Roi.Y1':[],'Roi.X2':[],'Roi.Y2':[],'ClassId':[]}
  for i in df.iterrows():
    for j in range(8):
      dk[cols[j]].append(i[1][0].split(';')[j])
  pd.DataFrame(dk).to_csv(name,index=False)

In [5]:
for en,path in enumerate(glob.glob('/content/GTSRB/Final_Training/Images/*/GT-*.csv')):
  df=pd.read_csv(path)
  if en==0:
    fnl=df
  else:
    fnl=pd.concat([df,fnl],0)
fnl['x_diff']=fnl['Roi.X2']-fnl['Roi.X1']
fnl['y_diff']=fnl['Roi.Y2']-fnl['Roi.Y1']
fnl['x_diff'].median(),fnl['y_diff'].median()

(33.0, 32.0)

In [7]:

for clas in tqdm(os.listdir('/content/GTSRB/Final_Training/Images/')):
  df=pd.read_csv('/content/GTSRB/Final_Training/Images/'+clas+'/GT-'+clas+'.csv')
  for i in df.iterrows():
    val=i[1].values
    name=val[0]
    x1=val[3]
    x2=val[5]
    y1=val[4]
    y2=val[6]
    total_name='/content/GTSRB/Final_Training/Images/'+clas+'/'+name
    img=cv2.imread(total_name)
    cv2.imwrite(total_name,cv2.resize(img[x1:x2,y1:y2,:],(30,30), interpolation = cv2.INTER_AREA))

100%|██████████| 43/43 [00:13<00:00,  3.31it/s]


In [41]:

dk={}
for en,clas in enumerate(os.listdir('/content/GTSRB/Final_Training/Images/')):
  dk[clas]={'train':[],'test':[],'validation':[]}
  ls=glob.glob('/content/GTSRB/Final_Training/Images/'+clas+'/*_00000.ppm')
  ln=len(ls)
  all=list(range(ln))
  test_val=round(0.2*ln)
  
  
  dk[clas]['test']=random.sample(all, test_val)
  for i in dk[clas]['test']:
    all.remove(i)

  dk[clas]['validation']=random.sample(all, test_val)
  for i in dk[clas]['validation']:
    all.remove(i)
  

  dk[clas]['train']=all
